<a href="https://colab.research.google.com/github/pawan-nandakishore/image_emotion_recognition/blob/master/three_model_-ensemble2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.python.lib.io import file_io
from skimage.transform import rescale, resize

physical_devices = tf.config.experimental.list_physical_devices('GPU')
if physical_devices:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

print(physical_devices)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive



From Turker's ensemble training it seems like the top three models are:

 - ResNet-BEST-73.2.h5 : 73.5 % <br>
 - RESNET50-AUX-BEST-72.7.h5:  72.5 % <br>
 - SENET50-AUX-BEST-72.5.h5: 72.5 % 
The attempt will be to combine these three models and see what type of accuracies one can get.

In [ ]:

# Function that reads the data from the csv file, increases the size of the images and returns the images and their labels
# dataset: Data path
IMAGE_SIZE = 48

def get_data(dataset):
    
    file_stream = file_io.FileIO(dataset, mode='r')
    data = pd.read_csv(file_stream)
    data[' pixels'] = data[' pixels'].apply(lambda x: [int(pixel) for pixel in x.split()])
    X, Y = data[' pixels'].tolist(), data['emotion'].values
    X = np.array(X, dtype='float32').reshape(-1,IMAGE_SIZE, IMAGE_SIZE,1)
    X = X/255.0
   
    X_res = np.zeros((X.shape[0], Resize_pixelsize,Resize_pixelsize,3))
    for ind in range(X.shape[0]): 
        sample = X[ind]
        sample = sample.reshape(IMAGE_SIZE, IMAGE_SIZE)
        image_resized = resize(sample, (Resize_pixelsize, Resize_pixelsize), anti_aliasing=True)
        X_res[ind,:,:,:] = image_resized.reshape(Resize_pixelsize,Resize_pixelsize,1)

    Y_res = np.zeros((Y.size, 7))
    Y_res[np.arange(Y.size),Y] = 1    
    
    return  X, X_res, Y_res

In [ ]:
Resize_pixelsize = 197

dev_dataset_dir = '/content/drive/My Drive/Personal projects/emotion_recognition_paper/data/fer_csv/dev.csv'
test_dataset_dir = '/content/drive/My Drive/Personal projects/emotion_recognition_paper/data/fer_csv/test.csv'

X_dev, X_res_dev, Y_dev   = get_data(dev_dataset_dir)
X_test, X_res_test, Y_test   = get_data(test_dataset_dir)


## Model 1: ACC 72.5%

In [ ]:
Senet_model_AUX = tf.keras.models.load_model('/content/drive/My Drive/Personal projects/emotion_recognition_paper/models/SENET50-AUX-BEST-72.5.h5')


In [ ]:
print('\n# Evaluate on dev data')
results_dev =Senet_model_AUX.evaluate(X_res_dev,Y_dev)
print('dev loss, dev acc:', results_dev)

print('\n# Evaluate on test data')
results_test = Senet_model_AUX.evaluate(X_res_test,Y_test)
print('test loss, test acc:', results_test)



# Evaluate on dev data
113/113 [==============================] - 6s 54ms/step - loss: 2.0713 - accuracy: 0.7108
dev loss, dev acc: [2.07125186920166, 0.7107829451560974]

# Evaluate on test data
113/113 [==============================] - 6s 53ms/step - loss: 1.8851 - accuracy: 0.7219
test loss, test acc: [1.8850795030593872, 0.7219281196594238]


## Model 2: ACC 73.2%

In [ ]:
Resnet_model = tf.keras.models.load_model('/content/drive/My Drive/Personal projects/emotion_recognition_paper/models/ResNet-BEST-73.2.h5')

In [ ]:
print('\n# Evaluate on dev data')
results_dev = Resnet_model.evaluate(X_res_dev,Y_dev)
print('dev loss, dev acc:', results_dev)

print('\n# Evaluate on test data')
results_test = Resnet_model.evaluate(X_res_test,Y_test)
print('test loss, test acc:', results_test)


# Evaluate on dev data
113/113 [==============================] - 5s 42ms/step - loss: 1.0431 - accuracy: 0.6960
dev loss, dev acc: [1.0430926084518433, 0.6960155963897705]

# Evaluate on test data
113/113 [==============================] - 5s 42ms/step - loss: 0.9164 - accuracy: 0.7322
test loss, test acc: [0.9163703322410583, 0.7322373986244202]


## Model 3: ACC 72.7%

In [ ]:
Resnet_model_AUX = tf.keras.models.load_model('/content/drive/My Drive/Personal projects/emotion_recognition_paper/models/RESNET50-AUX-BEST-72.7.h5')

In [ ]:
print('\n# Evaluate on dev data')
results_dev = Resnet_model_AUX.evaluate(X_res_dev,Y_dev)
print('dev loss, dev acc:', results_dev)

print('\n# Evaluate on test data')
results_test = Resnet_model_AUX.evaluate(X_res_test,Y_test)
print('test loss, test acc:', results_test)


# Evaluate on dev data
113/113 [==============================] - 5s 42ms/step - loss: 0.8615 - accuracy: 0.7074
dev loss, dev acc: [0.861474335193634, 0.7074394226074219]

# Evaluate on test data
113/113 [==============================] - 5s 42ms/step - loss: 0.8027 - accuracy: 0.7194
test loss, test acc: [0.8026625514030457, 0.7194204330444336]


In [ ]:
# make an ensemble prediction for multi-class classification
def ensemble_predictions( testX, models_TL, testresX):
    # make predictions
    yhats = np.zeros((len(models_TL),testX.shape[0],7))

    for model_ind in range(len(models_TL)):
        yhat = models_TL[model_ind].predict(testresX)
        yhats[model_ind,:,:] = yhat

    summed = np.sum(yhats, axis=0)
    result = np.argmax(summed, axis=1)
    return result
 
# evaluate a specific number of members in an ensemble
def evaluate_n_members(testX, models_TL, testresX, testy):
    # select a subset of members
    #subset = members[:n_members]
    #print(len(subset))
    # make prediction
    yhat = ensemble_predictions(testX, models_TL, testresX)
    # calculate accuracy
    return accuracy_score(testy, yhat)

In [ ]:
models_TL = [Resnet_model, Resnet_model_AUX, Senet_model_AUX]

ens_acc = evaluate_n_members(X_test, models_TL, X_res_test, np.argmax(Y_test, axis=1))
print(ens_acc)


0.7509055447199777
